In [ ]:
import boto3
import json

sqs = boto3.client('sqs')
s3 = boto3.client('s3')

# Implement this as a lambda function with a manual trigger - 'Start Pipeline?'

bucket_name = "abcd-v51"
prefix = "fmriresults01/abcd-mproc-release5"
queue_url = "https://sqs.us-east-2.amazonaws.com/575108944090/cloudpipe-jobqueue.fifo"
# def lambda_handler(event, context):
subjids = []

def fill_queue():

    # List objects in the S3 bucket
    paginator = s3.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=prefix)
    for page in page_iterator:
        if 'Contents' in page:
            for obj in page['Contents']:
                print(obj)
                if 'baselineYear1Arm1_ABCD-MPROC-T1' in obj['Key']:
                    print(obj['Key'])
                    prefix_path = obj['Key'].split('/')[-1]
                    subjid = prefix_path.split('_')[0]
                    subjids.append(subjid)

    queue_url = sqs.get_queue_url(QueueName='cloudpipe-jobqueue.fifo')['QueueUrl']
    id_list_length = len(subjids)
    mod_10 = id_list_length % 10
    iter_list = [*range(0, id_list_length - mod_10, 10), *range(id_list_length - mod_10, id_list_length)]
    for i in iter_list:
        queue_entries = []
        for j in range(10):
            if i + j < id_list_length:
                print(subjids[i + j][0])
                queue_entries.append({'Id': subjids[i + j][0], 'MessageBody': subjids[i + j][0], 'MessageGroupId': 'ABCDsubjsesIDs'})
        sqs.send_message_batch(QueueUrl=queue_url, Entries=queue_entries)


fill_queue()

{'Key': 'fmriresults01/abcd-mproc-release5/NDARINV003RTV85_baselineYear1Arm1_ABCD-MPROC-MID-fMRI_20181001111800.tgz', 'LastModified': datetime.datetime(2025, 4, 3, 12, 16, 42, tzinfo=tzutc()), 'ETag': '"603f00f47a16197a4188b17da20579ca"', 'ChecksumAlgorithm': ['CRC64NVME'], 'ChecksumType': 'FULL_OBJECT', 'Size': 676231480, 'StorageClass': 'STANDARD'}
